In [1]:
pip install pycryptodome


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.2 MB/s eta 0:00:00


In [14]:
#Encrypt data w AES using CTR and HMAC
#mode operation used : CTR
from Crypto.Cipher import AES #using pycryptodome library
from Crypto.Hash import HMAC, SHA256 #using HMAC SHA256 for data integrity
from Crypto.Random import get_random_bytes

data = 'secret data to transmit'.encode()

aes_key = get_random_bytes(16)
hmac_key = get_random_bytes(16)

cipher = AES.new(aes_key, AES.MODE_CTR)
ciphertext = cipher.encrypt(data)

hmac = HMAC.new(hmac_key, digestmod=SHA256)
tag = hmac.update(cipher.nonce + ciphertext).digest()

with open("encrypted.bin", "wb") as f:
    f.write(tag)
    f.write(cipher.nonce)
    f.write(ciphertext)

# Share securely aes_key and hmac_key with the receiver
# encrypted.bin can be sent over an unsecure channel

In [18]:
#Decrypt and verify data integrity with HMAC
#generating a ValueError exception when tampering is detected
import sys
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256

# Somehow, the receiver securely get aes_key and hmac_key
# encrypted.bin can be sent over an unsecure channel

with open("encrypted.bin", "rb") as f:
    tag = f.read(32)
    nonce = f.read(8)
    ciphertext = f.read()

try:
    hmac = HMAC.new(hmac_key, digestmod=SHA256)
    hmac.update(nonce + ciphertext)
    hmac.verify(tag)
except ValueError:
    print("The message was modified!")
    sys.exit(1)

cipher = AES.new(aes_key, AES.MODE_CTR, nonce=nonce)
message = cipher.decrypt(ciphertext)
print("Message:", message.decode())

Message: secret data to transmit


In [10]:
#Encrypt n authenticate data using OCB and no HMAC
#mode operation used : OCB
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

data = 'secret data to transmit'.encode()

aes_key = get_random_bytes(16)

cipher = AES.new(aes_key, AES.MODE_OCB)
ciphertext, tag = cipher.encrypt_and_digest(data)
assert len(cipher.nonce) == 15

with open("encrypted.bin", "wb") as f:
    f.write(tag)
    f.write(cipher.nonce)
    f.write(ciphertext)

# Share securely aes_key with the receiver
# encrypted.bin can be sent over an unsecure channel

In [11]:
#decrypt
import sys
from Crypto.Cipher import AES

# Somehow, the receiver securely get aes_key and hmac_key
# encrypted.bin can be sent over an unsecure channel

with open("encrypted.bin", "rb") as f:
    tag = f.read(16)
    nonce = f.read(15)
    ciphertext = f.read()

cipher = AES.new(aes_key, AES.MODE_OCB, nonce=nonce)
try:
    message = cipher.decrypt_and_verify(ciphertext, tag)
except ValueError:
    print("The message was modified!")
    sys.exit(1)

print("Message:", message.decode())

Message: secret data to transmit
